# Binned Calculations?

## Packages

In [131]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import dask
import os
import glob

## Functions

In [184]:
def normal_value_spatial(year, month_start, month_end, model):
    '''
    uses glob to get usable file paths which are then used in seasonal average function
    inputs:
    year, month_start, month_end, & nldas model
    outputs:
    scalar value which is average 
    '''
    files_arr = []
    
    for month_i in np.arange(month_start, month_end+1, 1):
        files = sorted(glob.glob(f"{sm_base_path}/{year}/NLDAS_{model}0125_H.A{year}{str(month_i).zfill(2)}*.nc"))
        for file_i in files:
            files_arr.append(file_i)
    
    ds = xr.concat([xr.open_dataset(f) for f in files_arr], dim="time")
    seasonal_mean = ds.mean(dim='time')
    return seasonal_mean
    

## dask

In [ ]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="10GiB",
    walltime="03:00:00",
)

cluster.scale(jobs=10) 

In [158]:
from dask.distributed import Client

client = Client(cluster)

In [159]:
client

<Client: 'tcp://146.186.150.11:41325' processes=10 threads=10, memory=100.00 GiB>

## Calculations

In [193]:
# both these have year then dday is formatted as NLDAS_FORA0125_H.A19791221_dday.nc
#                             sm is formatted as NLDAS_VIC0125_H.A19790328.nc
# april to september taken as growing season per haqiqi 2021

model = "VIC"
dday_base_path = "/storage/home/cta5244/work/pyWBM_yield_data/NCEPNARR_NLDAS_tmax_tmin/"
sm_base_path = "/storage/home/cta5244/work/pyWBM_yield_data/VIC_daily/"
start_year, end_year = 1979, 2025
month_start = 4 # april
month_end = 9 # september

In [194]:
results = []
for year in np.arange(start_year, end_year):
    delayed_task = dask.delayed(normal_value_spatial)(
                        year=year, 
                        month_start=month_start,
                        month_end=month_end,
                        model=model
                    )
    results.append(delayed_task)


In [195]:
ds_seasonal_mean_arr = dask.compute(*results)
data_concat = xr.concat(ds_seasonal_mean_arr, dim='time')
data_concat_timemean = data_concat.mean(dim='time')    # this is the mean we want! 

In [201]:
filepath_save = f"/storage/home/cta5244/work/avila_et_al_2025_pyWBM_yield/data/{model}_seasonal_average_alltime_average_soilmoisture.nc"

In [202]:
data_concat_timemean.to_netcdf(filepath_save)